# ਕਲਾਉਡ ਵਿੱਚ ਡਾਟਾ ਸਾਇੰਸ: "Azure ML SDK" ਦਾ ਤਰੀਕਾ

## ਪਰਿਚਯ

ਇਸ ਨੋਟਬੁੱਕ ਵਿੱਚ, ਅਸੀਂ ਸਿੱਖਾਂਗੇ ਕਿ Azure ML SDK ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਮਾਡਲ ਨੂੰ ਟ੍ਰੇਨ, ਡਿਪਲੌਇ ਅਤੇ ਕਨਜ਼ਿਊਮ ਕਿਵੇਂ ਕਰਨਾ ਹੈ।

ਪੂਰਵ-ਸ਼ਰਤਾਂ:
1. ਤੁਸੀਂ ਇੱਕ Azure ML ਵਰਕਸਪੇਸ ਬਣਾਇਆ ਹੈ।
2. ਤੁਸੀਂ [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) ਨੂੰ Azure ML ਵਿੱਚ ਲੋਡ ਕੀਤਾ ਹੈ।
3. ਤੁਸੀਂ ਇਸ ਨੋਟਬੁੱਕ ਨੂੰ Azure ML Studio ਵਿੱਚ ਅਪਲੋਡ ਕੀਤਾ ਹੈ।

ਅਗਲੇ ਕਦਮ ਹਨ:

1. ਮੌਜੂਦਾ ਵਰਕਸਪੇਸ ਵਿੱਚ ਇੱਕ ਐਕਸਪੇਰੀਮੈਂਟ ਬਣਾਓ।
2. ਇੱਕ ਕੰਪਿਊਟ ਕਲਸਟਰ ਬਣਾਓ।
3. ਡਾਟਾਸੈਟ ਲੋਡ ਕਰੋ।
4. AutoMLConfig ਦੀ ਵਰਤੋਂ ਕਰਕੇ AutoML ਨੂੰ ਕਨਫਿਗਰ ਕਰੋ।
5. AutoML ਐਕਸਪੇਰੀਮੈਂਟ ਚਲਾਓ।
6. ਨਤੀਜਿਆਂ ਦੀ ਜਾਂਚ ਕਰੋ ਅਤੇ ਸਭ ਤੋਂ ਵਧੀਆ ਮਾਡਲ ਪ੍ਰਾਪਤ ਕਰੋ।
7. ਸਭ ਤੋਂ ਵਧੀਆ ਮਾਡਲ ਨੂੰ ਰਜਿਸਟਰ ਕਰੋ।
8. ਸਭ ਤੋਂ ਵਧੀਆ ਮਾਡਲ ਨੂੰ ਡਿਪਲੌਇ ਕਰੋ।
9. ਐਂਡਪੋਇੰਟ ਨੂੰ ਕਨਜ਼ਿਊਮ ਕਰੋ।

## Azure Machine Learning SDK-ਵਿਸ਼ੇਸ਼ ਇੰਪੋਰਟ


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## ਵਰਕਸਪੇਸ ਸ਼ੁਰੂ ਕਰੋ  
ਸੰਰਖਿਤ ਕਨਫਿਗਰੇਸ਼ਨ ਤੋਂ ਵਰਕਸਪੇਸ ਆਬਜੈਕਟ ਸ਼ੁਰੂ ਕਰੋ। ਯਕੀਨੀ ਬਣਾਓ ਕਿ ਕਨਫਿਗ ਫਾਈਲ .\config.json 'ਤੇ ਮੌਜੂਦ ਹੈ।  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## ਐਜ਼ਰ ਐਮਐਲ ਪ੍ਰਯੋਗਸ਼ਾਲਾ ਬਣਾਓ

ਆਓ ਉਸ ਵਰਕਸਪੇਸ ਵਿੱਚ ਇੱਕ ਪ੍ਰਯੋਗਸ਼ਾਲਾ 'aml-experiment' ਨਾਮ ਦੀ ਬਣਾਈਏ ਜਿਸਨੂੰ ਅਸੀਂ ਹੁਣੇ ਸ਼ੁਰੂ ਕੀਤਾ ਹੈ।


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## ਕੰਪਿਊਟ ਕਲੱਸਟਰ ਬਣਾਓ  
ਤੁਹਾਨੂੰ ਆਪਣੇ AutoML ਰਨ ਲਈ ਇੱਕ [ਕੰਪਿਊਟ ਟਾਰਗੇਟ](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) ਬਣਾਉਣ ਦੀ ਲੋੜ ਹੋਵੇਗੀ।  


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## ਡਾਟਾ  
ਯਕੀਨੀ ਬਣਾਓ ਕਿ ਤੁਸੀਂ ਡਾਟਾਸੈੱਟ ਨੂੰ Azure ML 'ਤੇ ਅੱਪਲੋਡ ਕਰ ਦਿੱਤਾ ਹੈ ਅਤੇ ਕੁੰਜੀ ਦਾ ਨਾਮ ਡਾਟਾਸੈੱਟ ਦੇ ਨਾਮ ਨਾਲ ਇੱਕੋ ਜਿਹਾ ਹੈ।  


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## ਸਭ ਤੋਂ ਵਧੀਆ ਮਾਡਲ ਨੂੰ ਡਿਪਲੌਇ ਕਰੋ

ਹੇਠਾਂ ਦਿੱਤਾ ਕੋਡ ਚਲਾਓ ਤਾਂ ਜੋ ਸਭ ਤੋਂ ਵਧੀਆ ਮਾਡਲ ਨੂੰ ਡਿਪਲੌਇ ਕੀਤਾ ਜਾ ਸਕੇ। ਤੁਸੀਂ ਡਿਪਲੌਇਮੈਂਟ ਦੀ ਸਥਿਤੀ ਨੂੰ Azure ML ਪੋਰਟਲ ਵਿੱਚ ਦੇਖ ਸਕਦੇ ਹੋ। ਇਹ ਕਦਮ ਕੁਝ ਮਿੰਟ ਲੈ ਸਕਦਾ ਹੈ।


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## ਐਂਡਪੌਇੰਟ ਨੂੰ ਵਰਤੋ  
ਤੁਸੀਂ ਹੇਠਾਂ ਦਿੱਤੇ ਨਮੂਨੇ ਵਿੱਚ ਇਨਪੁਟਸ ਸ਼ਾਮਲ ਕਰ ਸਕਦੇ ਹੋ।


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**ਅਸਵੀਕਾਰਨਾ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦੀ ਕੋਸ਼ਿਸ਼ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਵਿੱਚ ਰੱਖੋ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁਚੱਜੇਪਣ ਹੋ ਸਕਦੇ ਹਨ। ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਇਸਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਅਧਿਕਾਰਤ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
